In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [114]:
path="C:\\dev\\telstra\\data\\"

trainFile="train.csv"
testFile="test.csv"
resourceFile="resource_type.csv"
eventTypeFile="event_type.csv"
logFeatureFile="log_feature.csv"
severityTypeFile="severity_type.csv"

test=pd.read_csv(filepath_or_buffer=path+testFile,delimiter=",",header=0)
train=pd.read_csv(filepath_or_buffer=path+trainFile,delimiter=",",header=0)

resource=pd.read_csv(filepath_or_buffer=path+resourceFile,delimiter=",",header=0)
event=pd.read_csv(filepath_or_buffer=path+eventTypeFile,delimiter=",",header=0)
feature=pd.read_csv(filepath_or_buffer=path+logFeatureFile,delimiter=",",header=0)
severity=pd.read_csv(filepath_or_buffer=path+severityTypeFile,delimiter=",",header=0)
join=pd.DataFrame({'id':[]})
datasets={'train':train,'test':test,'resource':resource,'event':event,'feature':feature,'severity':severity}
notCat=['id','volume']

In [115]:
#convert table of ids and classes to binary row feature vectors for each unique id
#takes a long time, thus once computed save results to csv file
for key, dataset in datasets.items():
    print('Dataset: '+key)
    tmp=pd.get_dummies(dataset)
    print('Finished creating dummies')
    tmp=tmp.groupby('id').sum()
    tmp['id']=tmp.index
    dataset=tmp
    datasets[key]=dataset

Dataset: feature
Finished creating dummies
Dataset: test
Finished creating dummies
Dataset: severity
Finished creating dummies
Dataset: event
Finished creating dummies
Dataset: train
Finished creating dummies
Dataset: resource
Finished creating dummies


In [116]:
#creates join dataset table that will store features for train and test datasets
datasets['train']['sample']='train'
datasets['test']['sample']='test'
datasets['test']['fault_severity']=np.nan
join=pd.concat([datasets['train'],datasets['test']],ignore_index=True)
#join=join.to_sparse()

In [117]:
#joins features from different datasets into one dataframe, features only for evaluated ids are extracted
#(i.e. some features may be not related to any id)
samples=['train','test']
for key, dataset in datasets.items():
    if key not in samples:
        join=pd.merge(join,dataset,on='id',how='left') #can use concat?

join=join.set_index(['sample',join.index])

In [118]:
join=join.fillna(value=0)

In [125]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

features=join.columns.values
features=features[2:]

train_features = join.loc['train'][features]
train_target = join.loc['train']['fault_severity']

test_features = join.loc['test'][features]

#Use csc_matrix?

# Fit the model to training data
clf = RandomForestClassifier(n_estimators=100)
#clf = LinearSVC()
#clf = MultinomialNB()
clf = clf.fit(train_features, train_target)

score = clf.score(train_features, train_target)
print("Mean accuracy of Random Forest: {0}".format(score))

test_response=clf.predict_proba(test_features)

Mean accuracy of Random Forest: 0.5875897574854356


In [126]:
from sklearn.metrics import log_loss
from sklearn.cross_validation import train_test_split

# Split 69-31 train vs test data
train_x, test_x, train_y, test_y = train_test_split(train_features, train_target, test_size=0.31, random_state=0)

clf = clf.fit(train_x, train_y)
predict_y = clf.predict_proba(test_x)

print ("LogLoss = %.2f" % (log_loss(test_y, predict_y)))

LogLoss = 2.20


In [26]:
#save predictions to required format
out=pd.DataFrame({'id':[],'predict_0':[],'predict_1':[],'predict_2':[]})
out['id']=test['id']
out['predict_0']=test_response.T[0]
out['predict_1']=test_response.T[1]
out['predict_2']=test_response.T[2]
out=out[['id','predict_0','predict_1','predict_2']]
meanOut.to_csv('predictions.csv',index=False)